<a href="https://colab.research.google.com/github/FredericoSRamos/Projeto-IA-Alura-Google/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
#!pip install -q -U google-generativeai
#!pip install pytube
#!pip install assemblyai

import google.generativeai as genai
import assemblyai as aai
import pandas as pd
import numpy as np

from pytube import YouTube
from google.colab import userdata

genai.configure (api_key = userdata.get ("SECRET-API-KEY")) #Substituir pela própria API do Google AI Studio

aai.settings.api_key = userdata.get ("AAI-API-KEY") #Substituir pela própria API do Assembly AI
transcriber = aai.Transcriber (config = aai.TranscriptionConfig (language_code = "pt"))

def download (link):
  yt = YouTube (link)
  audio = yt.streams.get_audio_only ()
  return audio.download ()

def transcribe_audio (file):
  transcript = transcriber.transcribe (file)
  return transcript.text

def embed_fn (text):
  return genai.embed_content (model = "models/embedding-001",
                              content = text,
                              task_type = "SEMANTIC_SIMILARITY")["embedding"]

def interpret_fn (text):
  response = genai.GenerativeModel ("models/gemini-1.5-pro-latest").generate_content (f"Interprete esse texto:\n{text}")
  return response.text

def video_relacionado (consulta, df):
  embedding_consulta = embed_fn (consulta)
  produtos_escalares = np.dot (np.stack (df["Embeddings"]), embedding_consulta)
  indice = np.argmax (produtos_escalares)
  return df.iloc [indice]

data = []

while True:
  link = input ("Digite o link do vídeo (ou 'fim' para encerrar):")
  if link.lower () == "fim":
    break

  try:
    audio_file = download (link)
  except:
    print ("Um erro aconteceu ao tentar baixar esse vídeo\n")
    continue

  try:
    transcript = transcribe_audio (audio_file)
  except:
    print ("Um erro aconteceu ao tentar transcrever esse vídeo\n")
    continue

  data.append({
              "Titulo": audio_file.split("/")[-1].removesuffix(".mp4"),
              "Link": link,
              "Transcrição": transcript,
              "Interpretação": interpret_fn (transcript),
              "Embeddings": embed_fn (transcript)})

df = pd.DataFrame (data)

while True:
  consulta = input ("Digite uma entrada para consultar (ou 'fim' para encerrar):")
  if consulta.lower () == "fim":
    break

  video = video_relacionado (consulta, df)

  print ("Vídeo relacionado:")
  video

KeyboardInterrupt: 